In [6]:
import pandas as pd
outputFilePath = "/Users/vcroopana/Downloads/Spring2020/BigData/Proj1_No_SQL/yelp_dataset/business/"
df = pd.read_csv(outputFilePath+'user_category'+".csv")
print('end')

end


In [14]:
import sklearn.preprocessing as pp
from scipy.sparse import csr_matrix 
import numpy as np
import scipy.linalg.blas
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarities(mat, userInd):

    row_normed_mat = pp.normalize(csr_matrix(mat), axis=1) # 1 => normalizes each row, doing this for cosine similarity

    maxInd =0
    #val = row_normed_mat.getrow(i) * row_normed_mat.T : gives business-business similarity values
    #val = scipy.linalg.blas.sgemm(alpha=1.0, a= row_normed_mat.getrow(userInd).toarray(), b= row_normed_mat.toarray(), trans_b=True)
    
    val = cosine_similarity(row_normed_mat.getrow(userInd), row_normed_mat)
    maxInd = np.argsort(val.flatten())[-2]
    return maxInd
    
def getMostSimilarUser(df, key, keyType ):
    user_id_series = df['user_id']  
    vals = df.drop(['user_id'], axis = 1).to_numpy(dtype= 'float32')
    if keyType == 'index':
        userId = user_id_series[key]
        userInd = key
    else:
        userId = key
        userInd = user_id_series[user_id_series == key].index.tolist()
        if len(my_list) == 0:
            return 'user not found in database'
        else:
            userInd = userInd[0]
            print(userInd)
    
    cosineSimIndex = cosine_similarities(vals, userInd)
    cosineSimUserId = user_id_series[cosineSimIndex]
    print("input user index: "+ str(userInd) +", user_id: "+ userId)
    print("most similar user index: " + str( cosineSimIndex) +", user_id: "+cosineSimUserId)

In [22]:
getMostSimilarUser(df, 100, 'index')

started
input user index: 100, user_id: oAOE4UAC5ZbAjEGBEMCb4g
most similar user index: 530273, user_id: N2fU8K-6YSlGoJVE_Z1UYg


In [23]:
#retrieve data from Cassandra DB:
from cassandra.cluster import Cluster
import pandas as pd

if __name__ == "__main__":
    cluster = Cluster(['127.0.0.1'], port = 9042)
    keyspace = 'yelp_weather_data'
    userkey = 100 # userid or index
    keytype = 'index' # index or value
    session = cluster.connect(keyspace, wait_for_all_pools=True) #json - name of keyspace
    query = "SELECT * from user_category"
    df = pd.DataFrame(list(session.execute(query)))
    print('Extracted Df from DB')
    
    getMostSimilarUser(df, userkey, keytype)
    

Extracted Df from DB
started
input user index: 100, user_id: tQqCwxLgYOrBgBf4CFZV7A
most similar user index: 236, user_id: S6boRo4sQ1onCOpxhWkobw
